<a href="https://colab.research.google.com/github/JyotikaJayani-08/ISRO-Hackathon/blob/main/Sample.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1.Input Text Preparation

In [1]:
import cv2

input_image_path = "path_to_input_image_chip.jpg"
input_image = cv2.imread(input_image_path, cv2.IMREAD_GRAYSCALE)

2. Preprocessing


In [ ]:
# Resize the input image to match the resolution of the LRO WAC mosaic
target_resolution = (desired_width, desired_height)
resized_image = cv2.resize(input_image, target_resolution, interpolation=cv2.INTER_LINEAR)

# Image enhancement (e.g., contrast adjustment)
enhanced_image = cv2.equalizeHist(resized_image)

3.Feature Extraction


In [ ]:

# Initialize feature detector (e.g., ORB)
orb = cv2.ORB_create()

# Detect keypoints and descriptors in the input image
keypoints1, descriptors1 = orb.detectAndCompute(enhanced_image, None)

# Load and process the LRO WAC mosaic image
lro_image_path = "path_to_lro_wac_mosaic.tif"
lro_image = cv2.imread(lro_image_path, cv2.IMREAD_GRAYSCALE)

# Detect keypoints and descriptors in the LRO WAC mosaic
keypoints2, descriptors2 = orb.detectAndCompute(lro_image, None)

4. Image Matching


In [ ]:
#4. Image Matching
# Use FLANN based matcher for finding matches
flann_index = 6  # LSH index for ORB
index_params = dict(algorithm=flann_index, table_number=6, key_size=12, multi_probe_level=1)
search_params = dict(checks=50)

flann = cv2.FlannBasedMatcher(index_params, search_params)
matches = flann.knnMatch(descriptors1, descriptors2, k=2)

# Apply ratio test to find good matches
good_matches = []
for m, n in matches:
    if m.distance < 0.7 * n.distance:
        good_matches.append(m)

4. Image Matching


In [ ]:
# Use FLANN based matcher for finding matches
flann_index = 6  # LSH index for ORB
index_params = dict(algorithm=flann_index, table_number=6, key_size=12, multi_probe_level=1)
search_params = dict(checks=50)

flann = cv2.FlannBasedMatcher(index_params, search_params)
matches = flann.knnMatch(descriptors1, descriptors2, k=2)

# Apply ratio test to find good matches
good_matches = []
for m, n in matches:
    if m.distance < 0.7 * n.distance:
        good_matches.append(m)
5. Georeferencing
# Extract location of good matches
src_pts = np.float32([keypoints1[m.queryIdx].pt for m in good_matches]).reshape(-1, 1, 2)
dst_pts = np.float32([keypoints2[m.trainIdx].pt for m in good_matches]).reshape(-1, 1, 2)

# Compute homography matrix
H, mask = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC, 5.0)

# Transform points to get the geographic location
height, width = resized_image.shape
pts = np.float32([[0, 0], [0, height - 1], [width - 1, height - 1], [width - 1, 0]]).reshape(-1, 1, 2)
dst = cv2.perspectiveTransform(pts, H)

5. Georeferencing


In [ ]:
# Extract location of good matches
src_pts = np.float32([keypoints1[m.queryIdx].pt for m in good_matches]).reshape(-1, 1, 2)
dst_pts = np.float32([keypoints2[m.trainIdx].pt for m in good_matches]).reshape(-1, 1, 2)

# Compute homography matrix
H, mask = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC, 5.0)

# Transform points to get the geographic location
height, width = resized_image.shape
pts = np.float32([[0, 0], [0, height - 1], [width - 1, height - 1], [width - 1, 0]]).reshape(-1, 1, 2)
dst = cv2.perspectiveTransform(pts, H)

6. Crater Identification


In [ ]:
import geopandas as gpd

# Load the lunar crater catalogue as a GeoDataFrame
crater_catalogue_path = "path_to_lunar_crater_catalogue.shp"
crater_catalogue = gpd.read_file(crater_catalogue_path)

# Convert the matched points to GeoDataFrame
matched_points = gpd.GeoDataFrame(geometry=gpd.points_from_xy(dst[:, 0, 0], dst[:, 0, 1]), crs="EPSG:4326")

# Perform spatial join to find the crater name
matched_crater = gpd.sjoin(matched_points, crater_catalogue, how="left", op="intersects")
crater_name = matched_crater["crater_name"].iloc[0] if not matched_crater.empty else "Unknown"

7. Output


In [ ]:
# Output the geographic coordinates and crater name
latitude = matched_points.geometry.y.iloc[0]
longitude = matched_points.geometry.x.iloc[0]

print(f"Crater Coordinates: Latitude {latitude}, Longitude {longitude}")
print(f"Crater Name: {crater_name}")